In [1]:
import numpy as np
import pandas as pd
import pypsa

%matplotlib inline

In [2]:
load = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='load', parse_dates=['t'], index_col='t',usecols='A:M').dropna(axis=1, how='all').dropna()
pv = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='pv', parse_dates=['t'], index_col='t',usecols='A:M').dropna(axis=1, how='all').dropna()
wind = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='wind', parse_dates=['t'], index_col='t',usecols='A:M').dropna(axis=1, how='all').dropna()
wind_offshore = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='wind_offshore', parse_dates=['t'], index_col='t',usecols='A:M').dropna(axis=1, how='all').dropna()
chp = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='chp', parse_dates=['t'], index_col='t',usecols='A:AD').dropna(axis=1, how='all').dropna()

buses = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='buses', index_col=0)
cbf_buses = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='cbf_buses', index_col=0)
lines = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='lines', index_col='name').dropna(axis=1, how='all').dropna()
links = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='links', index_col='name').dropna(axis=1, how='all').dropna()
gen_pv = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_pv', index_col=0)
gen_wind = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_wind', index_col=0)
gen_wind_offshore = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_wind_offshore', index_col=0)
gen_gas = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_gas', index_col=0, usecols='A:H')
gen_oil = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_oil', index_col=0)
gen_coal = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_coal', index_col=0)
gen_nuclear = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_nuclear', index_col=0)
gen_biomass = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_biomass', index_col=0, usecols='A:G')
gen_biogas = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_biogas', index_col=0, usecols='A:G')
gen_hydro = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_hydro', index_col=0)
gen_other_res = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_other_res', index_col=0)
gen_dsr = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_dsr', index_col=0, usecols='A:F')
gen_chp = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_chp', index_col=0, usecols='A:F')
gen_cbf = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='gen_cbf', index_col=0)
st_hps = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='st_hps', index_col=0)
st_battery = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='st_battery', index_col=0, usecols='A:I')
st_other = pd.read_excel('pypsa-uk-prototype.xlsx', sheet_name='st_other', index_col=0, usecols='A:I')

In [3]:
network = pypsa.Network()
network.set_snapshots(load.index)

network.import_components_from_dataframe(buses, 'Bus')
network.import_components_from_dataframe(cbf_buses, 'Bus')

network.import_components_from_dataframe(lines, 'Line')
network.lines['s_nom_extendable'] = True  

network.madd("Link", links.index,
             bus0=links['bus0'].tolist(), bus1= links['bus1'].tolist(),
             p_nom=0)#links['p_nom'].tolist())

network.madd("Generator", gen_cbf.index, suffix='_CBF', carrier='CBF',
             bus=gen_cbf.index.tolist(), p_nom=gen_cbf['p_nom'].to_list(), p_nom_extendable=False, 
             marginal_cost=gen_cbf['marginal_cost'].to_list())

dict_carriers = {
    'Lignite': 0.334, 'Hard coal': 0.354, 'Gas': 0.187, 'Gas CCS': 0, 'Oil': 0.334, 'Hydrogen': 0, 'Biomass': 0.403, 'Biogas': 0.178, 'BECCS': 0,
    'Geothermal': 0.026, 'Wind': 0, 'Wind offshore': 0, 'PV': 0, 'HPS': 0, 'Hydro': 0, 'Other RES': 0, 'CBF': 0, 'VOLL': 0, 'Battery': 0, 'Other storage' : 0, 'Nuclear': 0
}

for carrier in dict_carriers:
    network.add("Carrier", name=carrier, co2_emissions=dict_carriers[carrier])

network.madd("Load", load.columns, bus=load.columns, p_set=load)

C:\ProgramData\Miniconda3\envs\instrat-energy-model\lib\site-packages\pypsa\io.py:771: FutureWarning:

Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead



Index(['EN_NorthEast', 'EN_NorthWest', 'EN_Yorkshire', 'EN_EastMidlands',
       'EN_WestMidlands', 'EN_East', 'EN_London', 'EN_SouthEast',
       'EN_SouthWest', 'EN_Wales', 'Scotland', 'NorthernIreland'],
      dtype='object')

In [4]:
network.madd('Generator', 
      gen_pv['bus'],
      suffix='_PV',
      bus=gen_pv['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_pv['p_nom'].to_list(),
      carrier='PV', 
      marginal_cost=gen_pv['marginal_cost'].to_list(), 
      p_max_pu=pv)


network.madd('Generator', 
      gen_wind['bus'],
      suffix='_Wind',
      bus=gen_wind['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind['p_nom'].to_list(), 
      carrier='Wind', 
      marginal_cost=gen_wind['marginal_cost'].to_list(), 
      p_max_pu=wind)

network.madd('Generator', 
      gen_wind_offshore['bus'],
      suffix='_Wind_offshore',
      bus=gen_wind_offshore['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind_offshore['p_nom'].to_list(), 
      carrier='Wind offshore', 
      marginal_cost=gen_wind_offshore['marginal_cost'].to_list(), 
      p_max_pu=wind_offshore)

network.madd('Generator', 
      gen_gas.index,
      bus=gen_gas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_gas['p_nom'].to_list(), 
      carrier=gen_gas['carrier'].to_list(), 
      marginal_cost=gen_gas['marginal_cost'].to_list(),
      efficiency=gen_gas['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_oil.index,
      bus=gen_oil['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil['p_nom'].to_list(), 
      carrier='Oil', 
      marginal_cost=gen_oil['marginal_cost'].to_list(),
      efficiency=gen_oil['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal.index,
      bus=gen_coal['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal['p_nom'].to_list(), 
      carrier=gen_coal['carrier'].to_list(), 
      marginal_cost=gen_coal['marginal_cost'].to_list(),
      efficiency=gen_coal['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_nuclear.index,
      bus=gen_nuclear['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_nuclear['p_nom'].to_list(), 
      carrier='Nuclear', 
      marginal_cost=gen_nuclear['marginal_cost'].to_list(),
      efficiency=gen_nuclear['efficiency'].to_list(),
      p_max_pu=gen_nuclear['p_max_pu'].to_list(),
      p_min_pu=gen_nuclear['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_biomass.index,
      bus=gen_biomass['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biomass['p_nom'].to_list(), 
      carrier=gen_biomass['carrier'].to_list(), 
      marginal_cost=gen_biomass['marginal_cost'].to_list(),
      efficiency=gen_biomass['efficiency'].to_list(),
      p_max_pu=gen_biomass['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_biogas['bus'],
      suffix='_Biogas',
      bus=gen_biogas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biogas['p_nom'].to_list(), 
      carrier='Biogas', 
      marginal_cost=gen_biogas['marginal_cost'].to_list(),
      efficiency=gen_biogas['efficiency'].to_list(),
      p_max_pu=gen_biogas['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_hydro['bus'],
      suffix='_Hydro',
      bus=gen_hydro['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_hydro['p_nom'].to_list(), 
      carrier='Hydro', 
      marginal_cost=gen_hydro['marginal_cost'].to_list(),
      p_max_pu=gen_hydro['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_other_res['bus'],
      suffix='_OtherRES',
      bus=gen_other_res['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_other_res['p_nom'].to_list(), 
      carrier='Other RES', 
      marginal_cost=gen_other_res['marginal_cost'].to_list(),
      p_max_pu=gen_other_res['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_dsr['bus'],
      suffix='_DSR',
      bus=gen_dsr['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_dsr['p_nom'].to_list(), 
      carrier='DSR', 
      marginal_cost=gen_dsr['marginal_cost'].to_list(),
      p_max_pu=gen_dsr['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_chp.index,
      bus=gen_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_chp['p_nom'].to_list(), 
      carrier=gen_chp['carrier'].to_list(), 
      marginal_cost=gen_chp['marginal_cost'].to_list(),
      p_max_pu=chp,
      p_min_pu=chp*0.9,
      efficiency=gen_chp['efficiency'].to_list()
      )

network.madd("StorageUnit", st_hps.index, bus=st_hps['bus'].tolist(), carrier=st_hps['carrier'].tolist(),
             p_nom=st_hps['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_hps['max_hours'].to_list(),
             p_max_pu=st_hps['p_max_pu'].tolist(),
             efficiency_dispatch=st_hps['efficiency_dispatch'].tolist(),
             standing_loss=st_hps['standing_loss'].tolist())
            
network.madd("StorageUnit", st_battery.index, bus=st_battery['bus'].tolist(), carrier=st_battery['carrier'].tolist(),
             p_nom=st_battery['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_battery['max_hours'].to_list(),
             p_max_pu=st_battery['p_max_pu'].tolist(),
             efficiency_dispatch=st_battery['efficiency_dispatch'].tolist(),
             standing_loss=st_battery['standing_loss'].tolist())

network.madd("StorageUnit", st_other.index, bus=st_other['bus'].tolist(), carrier=st_other['carrier'].tolist(),
             p_nom=st_other['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_other['max_hours'].to_list(),
             p_max_pu=st_other['p_max_pu'].tolist(),
             efficiency_dispatch=st_other['efficiency_dispatch'].tolist(),
             standing_loss=st_other['standing_loss'].tolist())


Index(['EN_NorthEast_OtherStorage', 'EN_NorthWest_OtherStorage',
       'EN_Yorkshire_OtherStorage', 'EN_EastMidlands_OtherStorage',
       'EN_WestMidlands_OtherStorage', 'EN_East_OtherStorage',
       'EN_London_OtherStorage', 'EN_SouthEast_OtherStorage',
       'EN_SouthWest_OtherStorage', 'EN_Wales_OtherStorage',
       'Scotland_OtherStorage', 'NorthernIreland_OtherStorage'],
      dtype='object', name='name')

In [5]:
network.lopf(snapshots=network.snapshots, solver_name='cplex', pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
C:\ProgramData\Miniconda3\envs\instrat-energy-model\lib\site-packages\pypsa\linopf.py:85: FutureWarning:

Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead

C:\ProgramData\Miniconda3\envs\instrat-energy-model\lib\site-packages\pypsa\linopf.py:185: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

INFO:pypsa.linopf:Total preparation time: 22.49s
INFO:pypsa.linopf:Solve linear problem using Cplex solver


Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 5834160 rows and 1454178 columns.
Aggregator did 96360 substitutions.
Reduced LP has 315360 rows, 1340280 columns, and 2566651 nonzeros.
Presolve time = 9.41 sec. (3946.57 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =     284529180.000000
Perturbation started.
Iteration:   101   Dual objective     =     284529180.000000
Iteration:  2788   Dual objective     =     284529180.000000
Iteration:  5204   Dual objective     =     284614807.745137
Iteration:  7532   Dual objective     =     284762064.852298
Iteration:  9847   Dual objective     =     284860874.752940
Iteration: 12140   Dual objective     =     284873981.05

INFO:pypsa.linopf:Optimization successful. Objective value: 1.06e+09


('ok', 'optimal')

In [6]:
p_by_carrier = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum()

generation_by_fuel_yearly = p_by_carrier.sum()/1000000
generation_by_fuel_yearly

carrier
BECCS              5.654215
Biogas             4.160019
Biomass            9.946779
CBF                0.000000
DSR                0.004206
Gas                2.469590
Gas CCS            1.853793
Hard coal          0.000000
Hydro              1.244047
Hydrogen          12.161420
Nuclear           56.905836
Oil                0.026186
Other RES          9.675949
PV                44.233424
Wind              66.060977
Wind offshore    145.330193
dtype: float64

In [29]:
domestic_generation_by_fuel_yearly = generation_by_fuel_yearly
domestic_generation_by_fuel_yearly=domestic_generation_by_fuel_yearly.drop('CBF')
generation_by_fuel_yearly_percentage = domestic_generation_by_fuel_yearly.div(domestic_generation_by_fuel_yearly.sum(), axis=0)
generation_by_fuel_yearly_percentage = generation_by_fuel_yearly_percentage.round(4)
generation_by_fuel_yearly_percentage = (generation_by_fuel_yearly_percentage * 100).astype(str) + '%'
generation_by_fuel_yearly_percentage


carrier
BECCS             1.5699999999999998%
Biogas                          1.16%
Biomass                         2.77%
DSR                              0.0%
Gas                             0.69%
Gas CCS                         0.52%
Hard coal                        0.0%
Hydro            0.35000000000000003%
Hydrogen                        3.38%
Nuclear                        15.82%
Oil                             0.01%
Other RES                       2.69%
PV                              12.3%
Wind              18.360000000000003%
Wind offshore     40.400000000000006%
dtype: object

In [13]:
capacity_by_fuel = network.generators.groupby("carrier")["p_nom"].sum()
capacity_by_fuel

carrier
BECCS              1100.000000
Biogas             1831.568670
Biomass            5244.882000
CBF              120000.000000
DSR                2000.000000
Gas               10876.070000
Gas CCS            2630.000000
Hard coal             0.000000
Hydro              1875.744658
Hydrogen           6486.750000
Nuclear            6838.000000
Oil                 140.000000
Other RES          2052.000000
PV                51329.301222
Wind              34054.070707
Wind offshore     54550.000000
Name: p_nom, dtype: float64

In [14]:
storage_capacity = network.storage_units.groupby("carrier")["p_nom"].sum()
storage_capacity

carrier
Battery          21631.544078
Hydro             4888.000000
Other storage     3758.000000
Name: p_nom, dtype: float64